In [2]:
import pandas as pd
import sqlite3

In [8]:
conn = sqlite3.connect("busdata_18.sqlite")
cur = conn.cursor()

In [3]:
trips = pd.read_csv("trips_clean.csv")

In [4]:
unique_LINEID_list = trips.LINEID.tolist()
print(unique_LINEID_list)
print()
print("unique_LINEID_list length:", len(unique_LINEID_list))

['68', '25B', '45A', '25A', '14', '77A', '39', '16', '40D', '27B', '142', '83', '130', '15', '46A', '33', '7', '39A', '49', '1', '123', '41', '67X', '59', '9', '40', '239', '76', '84', '53', '185', '151', '13', '15B', '65B', '29A', '61', '140', '79A', '38A', '31', '33B', '69', '44', '42', '67', '184', '238', '145', '17A', '32', '27A', '17', '27X', '18', '122', '54A', '66', '150', '56A', '37', '27', '15A', '65', '11', '47', '79', '83A', '63', '4', '120', '41C', '70', '84A', '220', '39X', '32X', '68A', '84X', '38', '102', '270', '51X', '33X', '75', '26', '66A', '31A', '111', '14C', '114', '76A', '44B', '161', '7A', '43', '25', '104', '33A', '16C', '42D', '31B', '66X', '31D', '33D', '41B', '40B', '7D', '46E', '38D', '118', '51D', '15D', '41A', '25D', '66B', '38B', '236', '7B', '41X', '69X', '68X', '25X', '40E', '70D', '116', '77X', '16D', '33E', '41D']

unique_LINEID_list length: 130


In [5]:
def check_leavetimes_for_missing_data(unique_LINEID_list):
    for line in unique_LINEID_list:
        trips_result = cur.execute("SELECT * FROM trips_cleaned WHERE LINEID=?;", (line,))
        trips_df = pd.DataFrame(trips_result.fetchall(), columns=["DAYOFSERVICE", "TRIPID", "LINEID", "ROUTEID", "DIRECTION", "PLANNEDTIME_ARR", "PLANNEDTIME_DEP", "ACTUALTIME_ARR", "ACTUALTIME_DEP"])
        trips_df = trips_df.rename(columns={"PLANNEDTIME_ARR":"PLANNEDARR_TRIP", "PLANNEDTIME_DEP":"PLANNEDDEP_TRIP", "ACTUALTIME_ARR":"ACTUALARR_TRIP", "ACTUALTIME_DEP":"ACTUALDEP_TRIP"})
        unique_ROUTEID_list = trips_df.ROUTEID.unique().tolist()
        print("Trips dataframe created for", line)

        leavetimes_df =  pd.DataFrame(columns = ["DAYOFSERVICE", "TRIPID", "PROGRNUMBER", "STOPPOINTID", "PLANNEDTIME_ARR", "PLANNEDTIME_DEP", "ACTUALTIME_ARR", "ACTUALTIME_DEP"])

        unique_TRIPID_list = trips_df.TRIPID.unique().tolist()
        for trip in unique_TRIPID_list:
            result = cur.execute("SELECT DAYOFSERVICE, TRIPID, PROGRNUMBER, STOPPOINTID, PLANNEDTIME_ARR, PLANNEDTIME_DEP, ACTUALTIME_ARR, ACTUALTIME_DEP FROM leavetimes_cleaned WHERE TRIPID=?;", (trip,))
            current_subset = pd.DataFrame(result.fetchall(), columns=["DAYOFSERVICE", "TRIPID", "PROGRNUMBER", "STOPPOINTID", "PLANNEDTIME_ARR", "PLANNEDTIME_DEP", "ACTUALTIME_ARR", "ACTUALTIME_DEP"])
            leavetimes_df = leavetimes_df.append(current_subset)

        print("Leavetimes dataframe created for", line)
        leavetimes_df = leavetimes_df.rename(columns={"PLANNEDTIME_ARR":"PLANNEDARR_STOP", "PLANNEDTIME_DEP":"PLANNEDDEP_STOP", "ACTUALTIME_ARR":"ACTUALARR_STOP", "ACTUALTIME_DEP":"ACTUALDEP_STOP"})

        merged = leavetimes_df.merge(trips_df, on=["DAYOFSERVICE", "TRIPID"], how="left")
        dep_null = merged.loc[merged["ACTUALDEP_TRIP"].isna()]
        arr_null = merged.loc[merged["ACTUALARR_TRIP"].isna()]

        for route in unique_ROUTEID_list:
            current_route_df = merged.loc[merged["ROUTEID"] == route]
            PROGRNUMBER_list = current_route_df.PROGRNUMBER.unique().tolist()
            PROGRNUMBER_list.sort()
            first = dep_null.loc[(merged["ROUTEID"] == route) & (merged["PROGRNUMBER"] == PROGRNUMBER_list[0]) & (merged["ACTUALDEP_TRIP"].isna())]
            if first.shape[0] > 0:
                print("Imputable ACTUALTIME_DEP for", route + ":", first.shape[0])
            last = arr_null.loc[(merged["ROUTEID"] == route) & (merged["PROGRNUMBER"] == PROGRNUMBER_list[len(PROGRNUMBER_list)-1]) & (merged["ACTUALARR_TRIP"].isna())]
            if last.shape[0] > 0:
                print("Imputable ACTUALTIME_ARR for", route + ":", last.shape[0])
            print("Finished checking route", route)
        print("Completed", line)
        print("****************************************")
    

In [1]:
check_leavetimes_for_missing_data(unique_LINEID_list)